In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
f = open('tr')
raw = f.read().splitlines()
f.close()
print(raw[0])
print(raw[1])

In [ ]:
text = ''

for i in raw:
    text += re.sub('[^a-z ]', '', i.strip()[2:].lower())+' '
    
print(text)

word = list(set(nltk.word_tokenize(text)))
print(len(word))

In [ ]:
filtered = list(set([w for w in word if not w in stopwords.words('english')]))
print(len(filtered))
print(filtered)

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

wordnet_lemmatized_v = []

for item in filtered:
    wordnet_lemmatized_v.append(wordnet_lemmatizer.lemmatize(item, 'v'))

wordnet_v = list(set(wordnet_lemmatized_v))
print(len(wordnet_v))

In [ ]:
train_file_wordnet_v = []
Y_train_raw = []

for line in raw:
    Y_train_raw.append(int(line[0]))
    x = re.sub('[^a-z ]', '', line.strip()[2:].lower())
    tokens = nltk.word_tokenize(x)
    filtered_tokens = [w for w in tokens if not w in stopwords.words('english')]
    stemmed_tokens = ''
    for item in filtered_tokens:
        stemmed_tokens += wordnet_lemmatizer.lemmatize(item, 'v')+' '
    train_file_wordnet_v.append(stemmed_tokens[:-1])

print(len(train_file_wordnet_v))
print(train_file_wordnet_v[1])

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X_train_tfidf = vectorizer.fit_transform(train_file_wordnet_v)

In [ ]:
print(vectorizer.get_feature_names())
print(X_train_tfidf.shape)
print(type(X_train_tfidf))
X_train = X_train_tfidf.toarray()
Y_train = np.array(Y_train_raw)
print(type(X_train))
print(X_train.shape)

In [ ]:
test_file_wordnet_v = []
Y_test_raw = []

f = open('te')
read_test = f.read().splitlines()
f.close()

for line in read_test:
    Y_test_raw.append(int(line[0]))
    x = re.sub('[^a-z ]', '', line.strip()[2:].lower())
    tokens = nltk.word_tokenize(x)
    filtered_tokens = [w for w in tokens if not w in stopwords.words('english')]
    stemmed_tokens = ''
    for item in filtered_tokens:
        stemmed_tokens += wordnet_lemmatizer.lemmatize(item, 'v')+' '
    test_file_wordnet_v.append(stemmed_tokens[:-1])

print(len(test_file_wordnet_v))
print(test_file_wordnet_v[4])

In [ ]:
X_test_tfidf = vectorizer.transform(test_file_wordnet_v)
print(X_test_tfidf.shape)
print(type(X_test_tfidf))
X_test = X_test_tfidf.toarray()
Y_test = np.array(Y_test_raw)
print(type(X_test))
print(X_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
model_parameters = {
        'n_estimators':[50, 100, 150, 200],
        'criterion':['gini', 'entropy'],
        'max_depth': [3, 5, 7, 9, 11]
    }

In [ ]:
model = RandomForestClassifier(random_state=1)
gscv = GridSearchCV(estimator=model, 
                    param_grid=model_parameters, 
                    cv=5, 
                    verbose=1, 
                    n_jobs=-1,
                    scoring='roc_auc')

gscv.fit(X_train, Y_train)

In [ ]:
print('The best parameter are -', gscv.best_params_)

In [ ]:
print(gscv.best_score_)
print(gscv.best_estimator_)
print(gscv.scorer_)

In [ ]:
print('AUC on test by gscv =', roc_auc_score(y_true=Y_test,
                                                        y_score=gscv.predict_proba(X_test)[:, 1]))